<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [6]</a>'.</span>

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import scrapbook as sb 
import pandas as pd
import numpy as np 
import seaborn as sns 
import numpy as np
from statistics import mean , median
import matplotlib.pyplot as plt

# Baseline 

In [3]:
books = sb.read_notebooks("./BaseLine_Model_Output")
baseLine_data = []
for nb in books.notebooks: 
    nbList=[nb.scraps['Stats Model MAE'].data,nb.scraps['Catboost MAE'].data,nb.scraps['TabNet MAE'].data]
    baseLine_data.append(nbList)
df = pd.DataFrame(baseLine_data, columns = ["Stats Model","Catboost", "TabNet"])
baseline_data = np.array(baseLine_data)
stats = median(baseline_data[:,0])
catboost = median(baseline_data[:,1])
tabnet = median(baseline_data[:,2])
print(stats)
print(catboost)
print(tabnet)

0.20085925355447606
0.02016716867230926
0.05436785171892624


# GAN Model

In [4]:
book = sb.read_notebooks("./GAN_Output")
gan_data = []
gan_mse = []
for nb in book.notebooks:
    metrics = nb.scraps['GAN_1 Metrics'].data
    for i in range(1000):
        gan_mse.append(metrics[0][i])
    nbList = [nb.scraps['GAN Model MSE'].data,
              nb.scraps['GAN Model MAE'].data,
              nb.scraps['GAN Model Euclidean distance'].data,
              nb.scraps['GAN Model Manhattan Distance'].data]        
    gan_data.append(nbList)
    
df = pd.DataFrame(gan_data, columns = ['MSE','MAE','Euclidean Distance','Manhattan Distance'])
display(df.style)
print("MEDIAN:")
print(df.median(axis = 0))
gan_data = np.array(gan_data)
gan_median = median(gan_data[:,1])
print(gan_median)

,MSE,MAE,Euclidean Distance,Manhattan Distance
0,0.237758,0.289126,4.924161,29.490850
1,0.154385,0.177607,4.875983,27.351545
2,0.115848,0.166272,4.223787,25.605911
3,0.050628,0.130315,2.792225,20.068474
4,0.146849,0.170231,4.755492,26.215588
5,0.063462,0.137715,3.126184,21.208105
6,0.234429,0.297311,4.889643,30.325744
7,0.120596,0.227990,3.502377,23.255023
8,0.174217,0.286393,4.215143,29.212126
9,0.628786,0.388835,8.008329,39.661177


MEDIAN:
MSE                    0.150617
MAE                    0.202799
Euclidean Distance     4.489639
Manhattan Distance    26.783566
dtype: float64
0.20279892991634801


# GAN Model with TabNet generator

In [5]:
book = sb.read_notebooks("./GAN_Output")
gan_data = []
gan_mse = []
for nb in book.notebooks:
    metrics = nb.scraps["GAN_2 Metrics"].data
    for i in range(1000):
        gan_mse.append(metrics[0][i])
    nbList = [nb.scraps['GAN Model MSE'].data,
              nb.scraps['GAN Model MAE'].data,
              nb.scraps['GAN Model Euclidean distance'].data,
              nb.scraps['GAN Model Manhattan Distance'].data]        
    gan_data.append(nbList)
    
df = pd.DataFrame(gan_data, columns = ['MSE','MAE','Euclidean Distance','Manhattan Distance'])
display(df.style)
print("MEDIAN:")
print(df.median(axis = 0))
gan_data = np.array(gan_data)
gan_median = median(gan_data[:,1])
print(gan_median)

,MSE,MAE,Euclidean Distance,Manhattan Distance
0,0.237758,0.289126,4.924161,29.490850
1,0.154385,0.177607,4.875983,27.351545
2,0.115848,0.166272,4.223787,25.605911
3,0.050628,0.130315,2.792225,20.068474
4,0.146849,0.170231,4.755492,26.215588
5,0.063462,0.137715,3.126184,21.208105
6,0.234429,0.297311,4.889643,30.325744
7,0.120596,0.227990,3.502377,23.255023
8,0.174217,0.286393,4.215143,29.212126
9,0.628786,0.388835,8.008329,39.661177


MEDIAN:
MSE                    0.150617
MAE                    0.202799
Euclidean Distance     4.489639
Manhattan Distance    26.783566
dtype: float64
0.20279892991634801


# ABC_GAN Analysis

## ABC Pre-generator - Catboost 


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [6]:
book = sb.read_notebooks("./ABC_GAN_Stats")
paramVal = [[1,1],[1,0.1],[1,0.01],[1,0],[0.1,1],[0.1,0.1],[0.1,0.01],[0.1,0],[0.01,1],[0.01,0.1],[0.01,0.01],[0.01,0]]
abc_mae = [[] for i in range(12)]
abc_tab_mae = [[] for i in range(12)]
abc_mae_skip = [[] for i in range(12)]
abc_tab_mae_skip = [[] for i in range(12)]
abc_mae_mean = [[] for i in range(12)]
abc_mae_skip_mean = [[] for i in range(12)]
abc_weights = [[] for i in range(12)]
prior_model = [[] for i in range(12)]
abc_pre_generator = [[] for i in range(12)]

for nb in book.notebooks:
    # metrics1 = np.array(nb.scraps['ABC_GAN_1 Metrics'].data)
    metrics2 = np.array(nb.scraps['ABC_GAN_2 Metrics'].data)
    # metrics3 = np.array(nb.scraps['ABC_GAN_3 Metrics'].data)
    # metrics4 = np.array(nb.scraps['ABC_GAN_4 Metrics'].data)
    paramVar = float(nb.papermill_dataframe.iloc[0]['value'])
    paramBias = float(nb.papermill_dataframe.iloc[1]['value'])
    #Divide data according to parameters 
    for i in range(12):
        if paramVar == paramVal[i][0] and paramBias == paramVal[i][1]:
            for j in range(100):
                # abc_mae[i].append(metrics1[1,j])
                # abc_mae_skip[i].append(metrics3[1,j])
                abc_tab_mae[i].append(metrics2[1,j])
                # abc_tab_mae_skip[i].append(metrics4[1,j])
            # abc_weights[i].append(nb.scraps['Skip Connection Weight'].data)
            prior_model[i].append(nb.scraps['Prior Model MAE'].data)
            abc_pre_generator[i].append(nb.scraps['ABC Pre-generator MAE'].data)
            # abc_mae_skip_mean[i].append(mean(metrics3[1,:]))
            # abc_mae_mean[i].append(mean(metrics1[1,:]))
            abc_mae_mean[i].append(mean(metrics2[1,:]))

KeyError: 'Prior Model MAE'

In [ ]:

data = [[] for i in range(9)]
for i in range(9):
    for j in range(len(abc_weights[i])):
        data[i].append([paramVal[i][0], paramVal[i][1],prior_model[i][j],
                     abc_pre_generator[i][j],abc_weights[i][j],abc_mae_mean[i][j],abc_mae_skip_mean[i][j]])
    df = pd.DataFrame(data[i], columns = ['Variance','Bias','Prior Model MAE',
                                       'ABC pre-generator MAE','Skip Node weight','ABC GAN MAE','ABC_GAN MAE (skip connection)'])
    display(df.round(5))
    print(df.median(axis=0))
    print("-------------------------------------------------------------")

In [ ]:
# Display Catboost Summary Tables 
data = np.array(data)
catboostData = []
for i in range(9): 
    catboostData.append([gan_median,catboost, paramVal[i][0], paramVal[i][1],median(data[i][:,3]),median(data[i][:,5]),median(data[i][:,6]),median(data[i][:,4])])
df = pd.DataFrame(catboostData, columns = ['GAN','catboost','Variance','Bias','Prior Model MAE','mGAN','skipGAN','Skip Node weight'])
display(df.round(5))

## ABC Pre-generator - Stats 


In [ ]:
book = sb.read_notebooks("./ABC_GAN_Stats")
paramVal = [[1,1],[0.1,1],[0.01,1],[1,0.1],[0.1,0.1],[0.01,0.1],[1,0.01],[0.1,0.01],[0.01,0.01]]
abc_mae = [[] for i in range(9)]
abc_mae_skip = [[] for i in range(9)]
abc_mae_mean = [[] for i in range(9)]
abc_mae_skip_mean = [[] for i in range(9)]
abc_weights = [[] for i in range(9)]
prior_model = [[] for i in range(9)]
abc_pre_generator = [[] for i in range(9)]

for nb in book.notebooks:
    metrics1 = np.array(nb.scraps['ABC_GAN_1 Metrics'].data)
    metrics3 = np.array(nb.scraps['ABC_GAN_3 Metrics'].data)
    paramVar = float(nb.papermill_dataframe.iloc[0]['value'])
    paramBias = float(nb.papermill_dataframe.iloc[1]['value'])
    #Divide data according to parameters 
    for i in range(9):
        if paramVar == paramVal[i][0] and paramBias == paramVal[i][1]:
            for j in range(100):
                abc_mae[i].append(metrics1[1,j])
                abc_mae_skip[i].append(metrics3[1,j])
            abc_weights[i].append(nb.scraps['Skip Connection Weight'].data)
            prior_model[i].append(nb.scraps['Prior Model MSE'].data)
            abc_pre_generator[i].append(nb.scraps['ABC Pre-generator MSE'].data)
            abc_mae_skip_mean[i].append(mean(metrics3[1,:]))
            abc_mae_mean[i].append(mean(metrics1[1,:]))

In [ ]:
data = [[] for i in range(9)]
for i in range(9):
    for j in range(len(abc_weights[i])):
        data[i].append([paramVal[i][0], paramVal[i][1],prior_model[i][j],
                     abc_pre_generator[i][j],abc_weights[i][j],abc_mae_mean[i][j],abc_mae_skip_mean[i][j]])
    
    df = pd.DataFrame(data[i], columns = ['Variance','Bias','Prior Model MAE',
                                       'ABC pre-generator MAE','Skip Node weight','ABC GAN MAE','ABC_GAN MAE (skip connection)'])
    display(df.round(5))
    print(df.median(axis=0))
    print("-------------------------------------------------------------")

In [ ]:
# Display Stats Summary Tables 
statsData = []
data = np.array(data)
for i in range(9): 
    data[i] = np.array(data[i])
    statsData.append([gan_median,stats, paramVal[i][0], paramVal[i][1],median(data[i][:,3]),median(data[i][:,5]),median(data[i][:,6]),median(data[i][:,4])])
    #statsData.append([gan_median,stats, paramVal[i][0], paramVal[i][1]])
df = pd.DataFrame(statsData, columns = ['GAN','stats','Variance','Bias','Prior Model MAE','mGAN','skipGAN','Skip Node weight'])
display(df.round(5))